In [3]:
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
import implicit
import copy
from sklearn import preprocessing
from keras.models import load_model
import json

In [12]:
f = open('response.json')
data = json.load(f)
f.close()
print(data.keys())
print(len(data['data']))
data = data['data']

dict_keys(['data', 'summary'])
102080


In [23]:
# Вытащим уникальный список чеков и посмотрим длину
check_list_not_uniq = []
for item in data:
    check_list_not_uniq.append(item['UniqOrderId.Id'])

check_list = set(check_list_not_uniq)
print(len(check_list))

15259


In [19]:
# Вытащим словарь товаров и посмотрим длину
product_dict = {}
for item in data:
    product_dict[item['DishId']] = item['DishName']
print(len(product_dict))
    

326


In [76]:
# coберем словарик чеков в виде векторов

check_total = []
for check_num in check_list:
    check = {}
    check['id'] = check_num
    order = list(filter(lambda p: p['UniqOrderId.Id'] == check_num, data))
    list_product_order = []
    for pr in order:
        list_product_order.append(pr['DishId'])

    for product_key in product_dict.keys():
        if product_key in list_product_order:
            check[product_key] = 1
        else:
            check[product_key] = 0

    
    check_total.append(check)




In [78]:
columns = ['id'] + list(product_dict.keys())
print(columns[: 3])
print(len(columns))

['id', '8b618996-982c-4acf-84f7-400e7cff6cc7', '89254254-b5c6-4a29-ae67-fee29998a23e']
327


In [79]:
print(check_total[0])
print(len(check_total[0]))

{'id': 'ed9dac0c-8bd6-4c39-833c-7c08b3457f3d', '8b618996-982c-4acf-84f7-400e7cff6cc7': 0, '89254254-b5c6-4a29-ae67-fee29998a23e': 0, '92bf8ca1-8819-4d46-8039-b89a9df8dace': 0, '33cfb439-065a-45c4-8068-9575a8f69833': 0, '63018642-b3ce-47b0-a767-103b7ecc26d9': 0, 'ba33d2c0-2dba-4d22-bd5f-e6de73b77439': 1, '64ac0f42-cf07-4c23-afde-bf5e66747d4b': 0, '9b36cec1-e33e-4f99-bc8d-47bc7d5a63ed': 0, '7da2be02-1ed5-47b1-8774-e78637ccdd4d': 0, '21b939b5-8d53-4cb7-b4c4-879b421ce398': 0, '0751f186-2f45-47a4-bc1e-49716bc80203': 0, '98820e02-1489-4d48-a726-6a46036ec288': 0, 'b3c3dd5b-69d6-4133-8d0a-6ec318082421': 0, '20e9fe27-b115-4f30-95b3-dc6f53728567': 0, 'e4e52056-44fe-4e67-8342-b0e90ef8ce66': 0, '64656b73-340f-43c3-babf-d993337d74b5': 0, '8bf957a9-cf73-4079-8a16-ad39cb1bd88b': 0, '5985bea3-d017-4b74-828b-5888b88b1a33': 1, 'b2a73baa-088e-4ecc-9ee2-d4fb87de0776': 0, '23ce9d9f-b7a8-4f4e-a6e8-7630d0488f50': 0, 'cfb8c747-81e3-4ce9-83f3-916b5f2fcd2a': 0, '2443394a-d619-4c56-b3a7-739e770f2d94': 0, '6bf992

In [80]:

df = pd.DataFrame(data=check_total, columns=columns)
df.head()

,id,8b618996-982c-4acf-84f7-400e7cff6cc7,89254254-b5c6-4a29-ae67-fee29998a23e,92bf8ca1-8819-4d46-8039-b89a9df8dace,33cfb439-065a-45c4-8068-9575a8f69833,63018642-b3ce-47b0-a767-103b7ecc26d9,ba33d2c0-2dba-4d22-bd5f-e6de73b77439,64ac0f42-cf07-4c23-afde-bf5e66747d4b,9b36cec1-e33e-4f99-bc8d-47bc7d5a63ed,7da2be02-1ed5-47b1-8774-e78637ccdd4d,...,17ec39ee-006a-4b45-b108-06df8a45b11e,fe78aa4b-7acb-4459-81ec-f732d4db1ab2,f2196d2d-5f01-4b4b-9f9a-aded91d06a27,ffc2cc77-2b1c-42f3-a0db-e5992f69d140,642c015f-45e1-4e27-b316-a0e4c3ed2e38,9c7e670e-c6bc-4e0c-8fb7-f86d996c3812,03589208-ded0-49e0-a032-75df80464fc6,308ed4de-0dee-45e9-811c-7f2f80b1b55d,91a0d1e1-b7c6-432f-a5d4-b42bf8f112b6,599948b3-9dda-4b8d-af9d-853978205188
0,ed9dac0c-8bd6-4c39-833c-7c08b3457f3d,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,debf9663-1fcc-4712-83a1-f89985d40278,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,99cad92a-6ad6-47f7-b2a7-1610476bafc5,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,d6454496-cad0-418a-a74f-da94698fef4d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,c08e2e28-e3b0-45de-b3ea-335a3a986942,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
# Таким образом мы получили матрицу из 15259 (чеков) строк на 327 столбца (признака)
df.shape

(15259, 327)

In [84]:
# сохраним датафрейв в pikl чтобы не создавать его каждый раз
df.to_pickle('myDataFrame.pkl')

In [90]:
# Загрузим данные обратно из pickle
df2 = pd.read_pickle('myDataFrame.pkl')
df2.head()

,id,8b618996-982c-4acf-84f7-400e7cff6cc7,89254254-b5c6-4a29-ae67-fee29998a23e,92bf8ca1-8819-4d46-8039-b89a9df8dace,33cfb439-065a-45c4-8068-9575a8f69833,63018642-b3ce-47b0-a767-103b7ecc26d9,ba33d2c0-2dba-4d22-bd5f-e6de73b77439,64ac0f42-cf07-4c23-afde-bf5e66747d4b,9b36cec1-e33e-4f99-bc8d-47bc7d5a63ed,7da2be02-1ed5-47b1-8774-e78637ccdd4d,...,17ec39ee-006a-4b45-b108-06df8a45b11e,fe78aa4b-7acb-4459-81ec-f732d4db1ab2,f2196d2d-5f01-4b4b-9f9a-aded91d06a27,ffc2cc77-2b1c-42f3-a0db-e5992f69d140,642c015f-45e1-4e27-b316-a0e4c3ed2e38,9c7e670e-c6bc-4e0c-8fb7-f86d996c3812,03589208-ded0-49e0-a032-75df80464fc6,308ed4de-0dee-45e9-811c-7f2f80b1b55d,91a0d1e1-b7c6-432f-a5d4-b42bf8f112b6,599948b3-9dda-4b8d-af9d-853978205188
0,ed9dac0c-8bd6-4c39-833c-7c08b3457f3d,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,debf9663-1fcc-4712-83a1-f89985d40278,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,99cad92a-6ad6-47f7-b2a7-1610476bafc5,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,d6454496-cad0-418a-a74f-da94698fef4d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,c08e2e28-e3b0-45de-b3ea-335a3a986942,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
matrix = df2.drop('id', axis=1)
model = implicit.als.AlternatingLeastSquares(factors=327)
sparse_item = csr_matrix(matrix.values).tocsr()
model.fit(sparse_item)


100%|██████████| 15/15 [00:11<00:00,  1.26it/s]


In [92]:
idx = 2 # Найдем похожие на 2 чек
related = model.similar_items(idx, N=10)
related

[(2, 1.0),
 (2555, 0.6911632),
 (4791, 0.5658057),
 (6562, 0.56316745),
 (14074, 0.5534048),
 (1753, 0.5489883),
 (138, 0.528832),
 (14976, 0.52562326),
 (10943, 0.5226507),
 (11446, 0.5220333)]

In [101]:
df2.iloc[2]

pandas.core.series.Series

In [115]:
def dropZeroProduct(series, product_dict):
    product = [];
    for index, value in series.items():
        if value != 0:
            if index != 'id':
                product.append(product_dict[index])

    
    return product

In [116]:
# Наш основной товар
prod = dropZeroProduct(df2.iloc[2], product_dict)
prod

['Номер',
 'Острые',
 'Соус кетчуп (50 г)',
 'Токпоки 210гр',
 'Бекон (70 г)',
 'Ничего ',
 'Пуннопан с заварным кремом ',
 'Карбонара токпоки 210гр',
 'Корн-Дог с фри ',
 'Соус розовый (50 г) ',
 '12',
 'Чай Эрл Грей']

In [117]:
# Первый рекомендованный товар
prod2 = dropZeroProduct(df2.iloc[2555], product_dict)
prod2

['Номер',
 'Острые',
 'Соус чесночный (50 г)',
 'Токпоки 210гр',
 'Ничего ',
 'Корн - дог фри и сыром',
 'Соус розовый (50 г) ',
 '12',
 'Чай Эрл Грей']

In [130]:
prod3 = dropZeroProduct(df2.iloc[4791], product_dict)
prod3

['Номер',
 'Острые',
 'Токпоки 210гр',
 'Бекон (70 г)',
 'Карбонара токпоки 210гр',
 '4']

In [132]:
prod4 = dropZeroProduct(df2.iloc[6562], product_dict)
prod4

['Номер',
 'Острые',
 'Токпоки 210гр',
 'Бекон (70 г)',
 'Карбонара токпоки 210гр',
 '9']

In [133]:
prod5 = dropZeroProduct(df2.iloc[14074], product_dict)
prod5

['Номер',
 'Острые',
 'Токпоки 210гр',
 'Ничего ',
 'Кимпаб ',
 'Филе куриное (30 г)',
 'Соус сырный (50 г)',
 'Корн-Дог с фри ',
 '1',
 'Чай Эрл Грей']

In [135]:
set1 = {}
for item in related:
    pr = dropZeroProduct(df2.iloc[item[0]], product_dict)
    set1 = set(set1).union(set(pr))


In [136]:
#set1 = set(prod).union(set(prod2)).union(set(prod3)).union(set(prod4)).union(set(prod5))
set2 = set(prod)
list3 = list( set1.difference(set2) )
list3

['4',
 'Соус чесночный (50 г)',
 'Филе куриное (30 г)',
 '6',
 '9',
 '1',
 'Соус сырный (50 г)',
 'Корн - дог фри и сыром',
 'Кимпаб ',
 '14']